<a href="https://colab.research.google.com/github/chirag912/pepper_rp_2/blob/master/ChatBotResponse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# things we need for NLP
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import random
from tensorflow import keras
from keras.models import load_model

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Using TensorFlow backend.


In [0]:
# import our chat-bot intents file
import json
with open('/content/drive/My Drive/chatbot/intents.json') as json_data:
    intents = json.load(json_data)

In [5]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

36 documents
11 classes ['Phones', 'galaxy s10', 'goodbye', 'greeting', 'hours', 'iphone', 'noanswer', 'opentoday', 'payments', 'pixel', 'thanks']
54 unique stemmed words ["'s", 'a', 'acceiv', 'anyon', 'appl', 'ar', 'buy', 'bye', 'card', 'cash', 'credit', 'day', 'do', 'galaxy', 'good', 'goodby', 'googl', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'iphon', 'is', 'kind', 'lat', 'lik', 'mastercard', 'of', 'on', 'op', 'phon', 'pixel', 's10', 'samsung', 'see', 'smartphon', 'tak', 'thank', 'that', 'the', 'ther', 'to', 'today', 'want', 'what', 'when', 'would', 'x', 'xl', 'yo', 'you']


In [6]:
model = load_model('model_ChatBot.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [0]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.65
def classify(sentence):
    # generate probabilities from the model
    p = bow(sentence, words)
    
    d = len(p)
    f = len(documents)-2
    a = np.zeros([f, d])
    tot = np.vstack((p,a))
    
    results = model.predict(tot)[0]
    
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID, show_details=False):
    results = classify(sentence)
    print('Result:',results)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    #print(i)
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return (random.choice(i['responses']))
            results.pop(0)

In [9]:
response('i would like to buy a phone', '123', show_details=True)

Result: [('Phones', 0.9956216)]
context: buyflower
tag: Phones


'iphone, pixel and s10, which one do you like?'

In [10]:
context

{'123': 'buyflower'}

In [11]:
response('hello there', '123', show_details=True)

Result: [('greeting', 0.99999297)]
context: 
tag: greeting


'Hi there, how can I help?'

In [12]:
response('rose', '123', show_details=True)

Result: [('noanswer', 0.9962555)]
tag: noanswer


"Sorry, can't understand you"

In [13]:
context

{'123': ''}